# Concordância nas Anotações por Competência

In [1]:
# %%capture
# %pip install sklearn pandas plotly matplotlib fastparquet

## Importações

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score
from util import competence

# pd.options.plotting.backend = 'matplotlib'
pd.options.plotting.backend = 'plotly'

## Análise
As métricas usadas foram:
- Cohen's Kappa
- Percentual de igualdade entre as avaliações

Limpeza e Preprocessamento

In [3]:
df = competence.load_dataset(flat=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 375 entries, 93 to 882
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   (text, )         375 non-null    string
 1   (week, )         375 non-null    int64 
 2   (annot1, comp1)  375 non-null    int64 
 3   (annot1, comp2)  375 non-null    int64 
 4   (annot1, comp3)  375 non-null    int64 
 5   (annot1, comp4)  375 non-null    int64 
 6   (annot2, comp1)  375 non-null    int64 
 7   (annot2, comp2)  375 non-null    int64 
 8   (annot2, comp3)  375 non-null    int64 
 9   (annot2, comp4)  375 non-null    int64 
dtypes: int64(9), string(1)
memory usage: 32.2 KB


In [4]:
df.head()

text week annot1              \
                                                            comp1 comp2 comp3   
id                                                                              
93  Aconteceu uma coisa estava chuvendo umas chuva...    1      4     4     4   
94  nesse dia eu encontrei um diamante ele e muito...    1      3     3     3   
95  	Eu e Pedro estava em casa depois nos foi para...    1      2     3     3   
97  	Em um dia normal no meu quintal tava
chovendo...    1      4     3     4   
99  Aquele dia chovia muito, estava tendo muitos t...    1      4     3     3   

         annot2                    
   comp4  comp1 comp2 comp3 comp4  
id                                 
93     4      3     3     4     3  
94     3      3     2     3     2  
95     2      3     2     3     2  
97     4      3     3     3     3  
99     4      3     3     3     3

## Métricas Personalizadas

`aprox` = Métrica de Aproximação das avaliações

    {anot1: 5, anot2: 4} == 1.0
    {anot1: 5, anot2: 2} == 0.3333...


`vizinho` = % de avaliações com até 1 de distâcia.

    {anot1: 5, anot2: 4} == 1.0
    {anot1: 5, anot2: 3} == 0.0

In [5]:
@np.vectorize
def aprox(v1 ,v2):
    if v1 == v2:
        return 1.0
    """
    1 - (abs(5 - 1) - 1) / 3 == 0    # pior
    1 - (abs(5 - 4) - 1) / 3 == 1    # melhor
    """
    return 1 - (abs(v1 - v2) - 1) / 3

@np.vectorize
def vizinho(v1 ,v2):
    return float(abs(v1 - v2) < 2)

In [6]:
a = [5,5,5,5,5]
b = [5,4,3,2,1]

pd.DataFrame({
    'A': a, 'B': b,
    'Aproximação': aprox(a, b),
    'Vizinho Direto': vizinho(a, b)
})

,A,B,Aproximação,Vizinho Direto
0,5,5,1.000000,1.0
1,5,4,1.000000,1.0
2,5,3,0.666667,0.0
3,5,2,0.333333,0.0
4,5,1,0.000000,0.0


In [7]:
from typing import Callable
from functools import partial

def run_metrics(data: pd.DataFrame, metrics: dict[str, Callable]) -> pd.DataFrame:
    return pd.DataFrame({
        name: [
            func(data.annot1.comp1, data.annot2.comp1),
            func(data.annot1.comp2, data.annot2.comp2),
            func(data.annot1.comp3, data.annot2.comp3),
            func(data.annot1.comp4, data.annot2.comp4),
        ] for name, func in metrics.items()
    }, [f'Competência {i}' for i in range(1,5)])

analysis = partial(run_metrics, metrics={
    'Cohen`s Kappa': cohen_kappa_score,
    'Iguais': lambda v1, v2: np.equal(v1,v2).mean(),
    'Vizinho Direto': lambda v1, v2: vizinho(v1,v2).mean(),
    'Aproximação': lambda v1, v2: aprox(v1,v2).mean(),
})

In [8]:
g1 = df.groupby('week').get_group(1)

In [9]:
# cohen_kappa_score(g1.annot1.comp1, g1.annot2.comp1)
# from sklearn.metrics import confusion_matrix
g1.annot2.comp1.dtype
# confusion_matrix(g1.annot1.comp1, g1.annot2.comp1)

dtype('int64')

In [10]:
result = pd.concat({
    'Semana %d' % i: analysis(data) for i, data in df.groupby('week')
})
result

Cohen`s Kappa    Iguais  Vizinho Direto  Aproximação
Semana 1 Competência 1       0.308348  0.517241        0.931034     0.977011
         Competência 2       0.252818  0.448276        0.827586     0.919540
         Competência 3       0.010239  0.310345        0.758621     0.896552
         Competência 4       0.099379  0.310345        0.827586     0.942529
Semana 2 Competência 1       0.067653  0.265306        0.877551     0.959184
         Competência 2       0.125000  0.244898        0.612245     0.829932
         Competência 3      -0.064253  0.020408        0.306122     0.714286
         Competência 4       0.073043  0.265306        0.714286     0.897959
Semana 3 Competência 1       0.431433  0.560976        0.926829     0.975610
         Competência 2       0.378788  0.609756        0.902439     0.967480
         Competência 3       0.141796  0.243902        0.707317     0.878049
         Competência 4       0.287226  0.439024        0.902439     0.967480
Semana 4 Competência 1       0.361015  0.509091        0.927273     0.969697
         Competência 2       0.381733  0.563636        0.800000     0.933333
         Competência 3       0.115226  0.218182        0.509091     0.818182
         Competência 4       0.231086  0.381818        0.800000     0.921212
Semana 5 Competência 1       0.127669  0.301887        0.867925     0.955975
         Competência 2       0.163158  0.377358        0.735849     0.855346
         Competência 3      -0.039623  0.056604        0.584906     0.823899
         Competência 4       0.139684  0.283019        0.792453     0.930818
Semana 6 Competência 1       0.351323  0.500000        0.960000     0.986667
         Competência 2       0.399791  0.540000        0.840000     0.940000
         Competência 3       0.046381  0.120000        0.520000     0.806667
         Competência 4       0.186391  0.340000        0.900000     0.960000
Semana 7 Competência 1       0.095374  0.312500        0.937500     0.972222
         Competência 2       0.318777  0.458333        0.958333     0.986111
         Competência 3       0.086849  0.520833        0.958333     0.986111
         Competência 4       0.078067  0.354167        0.958333     0.986111
Semana 8 Competência 1       0.259520  0.580000        0.980000     0.993333
         Competência 2       0.246096  0.440000        0.960000     0.986667
         Competência 3       0.228706  0.540000        1.000000     1.000000
         Competência 4      -0.014085  0.280000        0.940000     0.980000

In [11]:
def plot(comp: str, name: str):
    return result.xs(comp, level=1).plot(title=comp + ' - ' + name).update_layout(xaxis_title='Semanas', yaxis_title='Percentual')

In [12]:
plot('Competência 1', 'Registro Formal')

In [13]:
plot('Competência 2', 'Coerência Temática')

In [14]:
plot('Competência 3', 'Tipologia Textual')

In [15]:
plot('Competência 4', 'Coesão')

## Criando Dataset de análise de discrepância
Obter anotações que mais discordam entre si usando a métrica de `vizinhos diretos`

Adicionando semana como coluna

In [16]:
discordances = competence.most_discrepant(df)
print(*discordances.keys())

comp1 comp2 comp3 comp4


In [17]:
discordances['comp1'].to_csv('competencia1.csv')
discordances['comp2'].to_csv('competencia2.csv')
discordances['comp3'].to_csv('competencia3.csv')
discordances['comp4'].to_csv('competencia4.csv')

In [18]:
!echo "Datasets das avaliações por competências com anotações que \ndivergem na classificação com mais de 1 nível de distancia" > info.txt
!zip data/apa-nlp-level-discordant.zip info.txt comp*.csv
!rm comp*.csv info.txt

  adding: info.txt (deflated 17%)
  adding: competencia1.csv (deflated 59%)
  adding: competencia2.csv (deflated 60%)
  adding: competencia3.csv (deflated 60%)
  adding: competencia4.csv (deflated 60%)


In [19]:
joined = competence.most_concordant_joined(df)
joined['comp1']

,week,level,text
id,,,
93,1,4,Aconteceu uma coisa estava chuvendo umas chuva...
94,1,3,nesse dia eu encontrei um diamante ele e muito...
95,1,3,Eu e Pedro estava em casa depois nos foi para...
97,1,4,Em um dia normal no meu quintal tava chovendo...
99,1,4,"Aquele dia chovia muito, estava tendo muitos t..."
...,...,...,...
878,8,3,"[T] a Menina rosinha, Era uma vez a menina El..."
879,8,3,[T] o ônibus pegou fogo 1 [P] o ônibus pegou f...
880,8,4,"[T]A borboleta [P]Era uma vez, uma linda lagar..."


In [20]:
joined['comp1'].to_csv('competencia1.csv')
joined['comp2'].to_csv('competencia2.csv')
joined['comp3'].to_csv('competencia3.csv')
joined['comp4'].to_csv('competencia4.csv')

In [21]:
Path('info.md').open('w').write("""\
# Tabelas de Anotação de Nível por competências filtrando as que mais concordam

As tabelas dos anotadores 1 e 2 foram unidas para cada competencia usando
como parametro de união a medida de `vizinho direto`.

```python
if abs(a,b) < 2:
    return max(a, b)
```

Estrutura dos datasets: **id, week, level, text**
""")
!zip data/apa-nlp-level.zip info.md competencia1.csv competencia2.csv competencia3.csv competencia4.csv
!rm info.md competencia1.csv competencia2.csv competencia3.csv competencia4.csv

NameError: name 'Path' is not defined